Referred 
https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb#scrollTo=ovpZyIhNIgoq

In [18]:
import tensorflow as tf
import numpy as np
import os
import time

#Added by Tomb 
import pathlib

In [19]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Added by Tomb
path_to_file = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/shakespeare.txt')
if not path_to_file.exists():
  tf.keras.utils.get_file(
      'shakespeare.txt', # your desired file name, you can even name it tomato.mid but the convention is to get the name from the end of the URL path
      origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [20]:
# Read then decode for py2 compat (python 2 compatibility)
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [21]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [22]:
# the unique characters in the file, meaning each character that is used 
vocab = sorted(set(text)) # return as a list
print ([text for text in enumerate(vocab)])
print(f'{len(vocab)} Unique characters')

[(0, '\n'), (1, ' '), (2, '!'), (3, '$'), (4, '&'), (5, "'"), (6, ','), (7, '-'), (8, '.'), (9, '3'), (10, ':'), (11, ';'), (12, '?'), (13, 'A'), (14, 'B'), (15, 'C'), (16, 'D'), (17, 'E'), (18, 'F'), (19, 'G'), (20, 'H'), (21, 'I'), (22, 'J'), (23, 'K'), (24, 'L'), (25, 'M'), (26, 'N'), (27, 'O'), (28, 'P'), (29, 'Q'), (30, 'R'), (31, 'S'), (32, 'T'), (33, 'U'), (34, 'V'), (35, 'W'), (36, 'X'), (37, 'Y'), (38, 'Z'), (39, 'a'), (40, 'b'), (41, 'c'), (42, 'd'), (43, 'e'), (44, 'f'), (45, 'g'), (46, 'h'), (47, 'i'), (48, 'j'), (49, 'k'), (50, 'l'), (51, 'm'), (52, 'n'), (53, 'o'), (54, 'p'), (55, 'q'), (56, 'r'), (57, 's'), (58, 't'), (59, 'u'), (60, 'v'), (61, 'w'), (62, 'x'), (63, 'y'), (64, 'z')]
65 Unique characters


## Vectorize the text

Before training, you need to convert the strings to a numerical representation.<br>

In [23]:
# It needs the text to be split into tokens first.
# the 'b' prefix printed out indicates that these characters are stored as bytes objects.
# When text data is read or processed in Python, especially with encoding methods like UTF-8, each character is represented as a byte in memory

example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # contains a tensor where each element corresponds to a token (character) from the input texts provided in the example_texts list.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [24]:
# Now create the tf.keras.layers.StringLookup layer:
# This is NOT zero index. It starts from 1. and the IDs given are unique indices from the vocab variable. "a"'s index is 39 in vaocab as zero index
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary = list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [65]:
# Invert back the vectors of IDs to Chars
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)
chars

<class 'keras.src.layers.preprocessing.string_lookup.StringLookup'>


<tf.Tensor: shape=(), dtype=string, numpy=b'i'>

In [26]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # chars_from_ids is an instance obj of tf.keras.layers.StringLookup

## Create Training Examples and Targets

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

In [27]:
"""First prepare IDs, then use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices."""
# tf.strings.unicode_split makes each chars in text tokens
# ids_from_chars variable has a class of tf.keras.layers.StringLookup in it which converts tokens into ids
# print("tokens!!>>>", tf.strings.unicode_split(text, 'UTF-8'))
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print("IDs!!>>>",all_ids)

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # ids_dataset becomes a class
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100

IDs!!>>> tf.Tensor([19 48 57 ... 46  9  1], shape=(1115394,), dtype=int64)
F
i
r
s
t
 
C
i
t
i


In [53]:
"""Convert these individual characters to sequences of the desired size"""
# The batch method lets you easily do that.
#  seq_length + 1, the model learns to predict the next character in the sequence given a fixed window of characters as input.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))


tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [42]:
# It's easier to see what this is doing if you join the tokens back into strings:
for seq in sequences.take(1):
    print(text_from_ids(seq).numpy()) # text_from_ids is the custom function

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. <br> At each time step the input is the current character and the label is the next character.

In [68]:
# Details below, but this function slides one char to right
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target) #  map automatically takes an argument. dataset is an instance obj of the _MapDataset class

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches
You used **tf.data** to split the text into manageable sequences. <br>
But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [69]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE) # This does Asynchronous data loading, i.e. improves the loading speed
)

#### ---Unused or test code below---

In [29]:
"""About tf.keras.layersStringLookup()"""

# vocab1 = ["a", "b", "c", "d"]
# data = tf.constant([["a", "c", "d"],["d", "z", "b"]])
# print(data)
# layer = tf.keras.layers.StringLookup(vocabulary=vocab1)
# layer(data)

# vocab = ['apple', 'banana', 'orange']
# string_lookup = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token='UNK') # UNK refers unknown and it gives the unknown element the index 0. StringLookup starts at 1 index

# ids = string_lookup(['apple', 'kiwi', 'orange'])
# print(ids.numpy())

'About tf.keras.layersStringLookup()'

In [30]:
"""about batch in tf.data.Dataset"""
# dataset = tf.data.Dataset.range(8)
# dataset = dataset.batch(3)
# print(list(dataset.as_numpy_iterator()))


# dataset1 = tf.data.Dataset.range(8)
# dataset1 = dataset1.batch(3, drop_remainder=True)
# print(list(dataset1.as_numpy_iterator()))

'about batch in tf.data.Dataset'

In [ ]:
"""About take() in tf.data.Dataset"""
# dataset = tf.data.Dataset.range(10)
# dataset = dataset.take(1)
# list(dataset.as_numpy_iterator())

# list(sequences.as_numpy_iterator())

# seq = sequences.take(1)
# list(seq.as_numpy_iterator())

In [57]:
"""How the split_input_target function works"""

# sequence = ["H", "e", "l", "l", "o"]
# sequence = list("Hello") # equivalent above but more convenient

# input, target = split_input_target(sequence)
# print(input)
# print(target)

['H', 'e', 'l', 'l']
['e', 'l', 'l', 'o']
